In [ ]:
# Cell 1: Setup and Load Data
import sys
sys.path.append('../src')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from preprocessor import DataPreprocessor
from data_loader import DataLoader
import warnings
warnings.filterwarnings('ignore')

# Load data
loader = DataLoader('../data/raw/historical_data.csv', '../data/raw/fear_greed_index.csv')
trader_data, sentiment_data = loader.load_all_data()
preprocessor = DataPreprocessor()

print("Original data shapes:")
print(f"Trader data: {trader_data.shape}")
print(f"Sentiment data: {sentiment_data.shape}")

# Cell 2: Preprocess Trader Data
print("=== PREPROCESSING TRADER DATA ===")
trader_processed = preprocessor.preprocess_trader_data(trader_data.copy())

print("New columns created:")
new_cols = set(trader_processed.columns) - set(trader_data.columns)
print(list(new_cols))

print("\nProcessed data info:")
print(trader_processed.info())

# Cell 3: Preprocess Sentiment Data
print("=== PREPROCESSING SENTIMENT DATA ===")
sentiment_processed = preprocessor.preprocess_sentiment_data(sentiment_data.copy())

print("New columns created:")
new_cols = set(sentiment_processed.columns) - set(sentiment_data.columns)
print(list(new_cols))

print("\nSentiment score mapping:")
print(sentiment_processed[['Classification', 'sentiment_score']].drop_duplicates())

# Cell 4: Handle Missing Values and Outliers
print("=== HANDLING MISSING VALUES AND OUTLIERS ===")

# Check for extreme outliers in PnL
q1 = trader_processed['closedPnL'].quantile(0.25)
q3 = trader_processed['closedPnL'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 3 * iqr
upper_bound = q3 + 3 * iqr

outliers = trader_processed[(trader_processed['closedPnL'] < lower_bound) | 
                          (trader_processed['closedPnL'] > upper_bound)]
print(f"Extreme PnL outliers found: {len(outliers)}")
print(f"Outlier range: {outliers['closedPnL'].min()} to {outliers['closedPnL'].max()}")

# Option to remove extreme outliers (uncomment if needed)
# trader_processed = trader_processed[(trader_processed['closedPnL'] >= lower_bound) & 
#                                   (trader_processed['closedPnL'] <= upper_bound)]

# Cell 5: Feature Engineering
print("=== FEATURE ENGINEERING ===")

# Create additional trading features
if 'time' in trader_processed.columns:
    trader_processed['time'] = pd.to_datetime(trader_processed['time'])
    trader_processed['hour'] = trader_processed['time'].dt.hour
    trader_processed['day_of_week'] = trader_processed['time'].dt.dayofweek
    trader_processed['month'] = trader_processed['time'].dt.month

# Risk metrics
if 'size' in trader_processed.columns and 'leverage' in trader_processed.columns:
    trader_processed['risk_exposure'] = trader_processed['size'] * trader_processed['leverage']

# Profit margin
if 'closedPnL' in trader_processed.columns and 'size' in trader_processed.columns:
    trader_processed['profit_margin'] = trader_processed['closedPnL'] / trader_processed['size']
    trader_processed['profit_margin'] = trader_processed['profit_margin'].replace([np.inf, -np.inf], 0)

print("New features created:")
feature_cols = ['hour', 'day_of_week', 'month', 'risk_exposure', 'profit_margin']
existing_features = [col for col in feature_cols if col in trader_processed.columns]
print(existing_features)

# Cell 6: Data Validation
print("=== DATA VALIDATION ===")

# Check data consistency
print("Data consistency checks:")
print(f"Date range: {trader_processed['date'].min()} to {trader_processed['date'].max()}")
print(f"PnL range: {trader_processed['closedPnL'].min()} to {trader_processed['closedPnL'].max()}")
print(f"Size range: {trader_processed['size'].min()} to {trader_processed['size'].max()}")

# Check for data quality issues
print(f"\nInfinite values: {np.isinf(trader_processed.select_dtypes(include=[np.number])).sum().sum()}")
print(f"Null values: {trader_processed.isnull().sum().sum()}")

# Cell 7: Merge Datasets
print("=== MERGING DATASETS ===")
merged_data = preprocessor.merge_datasets(trader_processed, sentiment_processed)

print(f"Merged data shape: {merged_data.shape}")
print(f"Successful merge rate: {(merged_data['Classification'].notna().sum() / len(merged_data) * 100):.1f}%")

# Check merge results
print("\nMerge summary:")
print(merged_data['Classification'].value_counts())

# Cell 8: Final Data Quality Check
print("=== FINAL DATA QUALITY CHECK ===")
print("Final dataset info:")
print(merged_data.info())

print("\nFinal missing values:")
print(merged_data.isnull().sum())

print("\nFinal data shape:", merged_data.shape)

# Cell 9: Visualize Preprocessing Results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Before/after PnL distribution
axes[0,0].hist(trader_data['closedPnL'].dropna(), bins=50, alpha=0.7, label='Original')
axes[0,0].hist(merged_data['closedPnL'].dropna(), bins=50, alpha=0.7, label='Processed')
axes[0,0].set_title('PnL Distribution: Before vs After')
axes[0,0].legend()

# Sentiment distribution in merged data
merged_data['Classification'].value_counts().plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Sentiment Distribution in Merged Data')
axes[0,1].tick_params(axis='x', rotation=45)

# Daily trade counts
if 'date' in merged_data.columns:
    daily_counts = merged_data.groupby('date').size()
    axes[1,0].plot(daily_counts.index, daily_counts.values)
    axes[1,0].set_title('Daily Trade Counts')
    axes[1,0].tick_params(axis='x', rotation=45)

# Missing data heatmap
missing_data = merged_data.isnull()
sns.heatmap(missing_data.iloc[:, :10], ax=axes[1,1], cbar=True)
axes[1,1].set_title('Missing Data Pattern (First 10 columns)')

plt.tight_layout()
plt.show()

# Cell 10: Save Processed Data
merged_data.to_csv('../data/processed/merged_data.csv', index=False)
trader_processed.to_csv('../data/processed/trader_data_processed.csv', index=False)
sentiment_processed.to_csv('../data/processed/sentiment_data_processed.csv', index=False)

print(" Preprocessing complete! All processed data saved.")
print(f"Final merged dataset: {merged_data.shape[0]} rows, {merged_data.shape[1]} columns")